In [1]:
import numpy as np

import pandas as pd
import pandas_ta as ta # pandas technical analysis
import pandas_datareader.data as web
pd.options.display.float_format = "{:,.2f}".format

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,6)
plt.style.use("classic")
plt.rcParams["font.size"] = 11
plt.rcParams["lines.linestyle"] = "-"
plt.rcParams["figure.dpi"] = 700

from datetime import datetime

import yfinance as yf

# momentum indicators
from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.momentum import StochasticOscillator
from ta.momentum import WilliamsRIndicator

# trend indicators
from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.trend import WMAIndicator
from ta.trend import SMAIndicator

# volatility indicators
from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange

# volume indicators
from ta.volume import money_flow_index
from ta.volume import OnBalanceVolumeIndicator

In [2]:
end = datetime.today()
start = datetime(end.year-1, end.month, end.day)

ticker = ["^GSPC"] # add tickers here

dataframe = web.DataReader(ticker, "yahoo", start, end)

In [3]:
dataframe = dataframe.stack() # https://medium.com/swlh/reshaping-in-pandas-with-stack-and-unstack-functions-bb169f64467d
dataframe = dataframe.reset_index() # not necessary, puts the same date to each innermost row index
dataframe

Attributes,Date,Symbols,Adj Close,Close,High,Low,Open,Volume
0,2020-09-16,^GSPC,"3,385.49","3,385.49","3,428.92","3,384.45","3,411.23",4710030000
1,2020-09-17,^GSPC,"3,357.01","3,357.01","3,375.17","3,328.82","3,346.86",4371940000
2,2020-09-18,^GSPC,"3,319.47","3,319.47","3,362.27","3,292.40","3,357.38",7068700000
3,2020-09-21,^GSPC,"3,281.06","3,281.06","3,285.57","3,229.10","3,285.57",4828350000
4,2020-09-22,^GSPC,"3,315.57","3,315.57","3,320.31","3,270.95","3,295.75",3963300000
...,...,...,...,...,...,...,...,...
247,2021-09-09,^GSPC,"4,493.28","4,493.28","4,529.90","4,492.07","4,513.02",3035300000
248,2021-09-10,^GSPC,"4,458.58","4,458.58","4,520.47","4,457.66","4,506.92",2851140000
249,2021-09-13,^GSPC,"4,468.73","4,468.73","4,492.99","4,445.70","4,474.81",3096390000
250,2021-09-14,^GSPC,"4,443.05","4,443.05","4,485.68","4,435.46","4,479.33",2568730000


In [4]:
class TechnicalIndicators:
    """
    This is an example of the TechIndicator class.
    """
    def __init__(self, symbol, date, close, high, low, volume):
        self.symbol = symbol
        self.date = date
        self.close = close
        self.high = high
        self.low = low
        self.volume = volume
        self.technical_indicators = None
        
    def set_technical_indicators(self):
        self.technical_indicators = pd.DataFrame()
       
        self.technical_indicators["Date"] = self.date
        self.technical_indicators["Close"] = self.close
        self.technical_indicators["High"] = self.high
        self.technical_indicators["Low"] = self.low
        self.technical_indicators["Volume"] = self.volume
        
# momentum indicators
    def rate_change(self, window):
        """
        Rate of Change. Momentum Indicator
        """
        closing_price = self.close
        df = self.technical_indicators
        
        roc = ROCIndicator(
                    close = closing_price,
                    window = window,
                    fillna = False
                    )
        
        df["ROC"] = roc.roc()

        return dataframe    
    
    def rsi(self, window):
        """
        The relative strength index. Momentum Indicator.
        """
        closing_price = self.close
        df = self.technical_indicators
        
        rsi = RSIIndicator(
                        close = closing_price, 
                        window = window,
                        fillna = False
                          )
        
        df["RSI"] = rsi.rsi()
        
        return dataframe
    
    def stochastic_oscillator(self, window):
        """
        Stochastic Oscillator. Momentum Indicator.
        """
        df = self.technical_indicators
        
        stoch_osci = StochasticOscillator(
            close = self.close,
            high = self.high,
            low = self.low,
            window = window,
            smooth_window = 3,
            fillna = False
             )
        
        df["Stochastic Oscillator"] = stoch_osci.stoch()
    
        return dataframe
    
    def WilliamsR(self, window):
        """
        WilliamsR Indicator. Momentum Indicator.
        """
        df = self.technical_indicators

        williamsR = WilliamsRIndicator(
                                    high = self.high,
                                    low = self.low,
                                    close = self.close,
                                    lbp = window,
                                    fillna = False
                                      )
    
        df["Williams %R Indicator"] = williamsR.williams_r()
    
        return dataframe
    
# trend indicators
    def moving_average(self):
        """
        Moving Average.
        """
        df = self.technical_indicators
        days = [5,10,20]
        
        for i in days:
            col_name = f"MA for {i} days"
            df[col_name] = self.price.rolling(window = i).mean()
            
        return dataframe
    
    def macd(self):
        """
        The moving average convergence/divergence.
        """
        df = self.technical_indicators
        closing_price = self.close
        ema_26 = 26
        ema_12 = 12
        signal = 9
        macd = MACD(
            closing_price, 
            window_fast = ema_12, 
            window_slow = ema_26,
            window_sign = signal,
            fillna = False
                    )
        df["MACD"] = macd.macd()
        
        return dataframe

    def ema(self, window):
        """
        The exponential moving average. Trend indicator.
        """
        df = self.technical_indicators
    
        ema = EMAIndicator(
                close = self.close,
                window = window,
                fillna = False
                    )
        
        df["Exponential moving average"] = ema.ema_indicator()
        
        return dataframe
    
    def wma(self, window):
        """
        Weighted moving average. Trend indicator.
        """
        df = self.technical_indicators
        
        wma = WMAIndicator(
                close = self.close, 
                window = window, 
                fillna = False
                )
        
        df["Weighted moving average"] = wma.wma()
        
        return dataframe

# volatility indicators
    def bollinger_bands(self, window):
        """
        Bollinger bands. Volatility Indicator.
        """
        closing_price = self.close
        df = self.technical_indicators
        
        bbands = BollingerBands(
                            close = closing_price,
                            window = window,
                            window_dev = 2,
                            fillna= False
                               )

        df["Bollinger Bands Middle"] = bbands.bollinger_mavg()
        df["Bollinger Bands Lower"] = bbands.bollinger_lband()
        df["Bollinger Bands Upper"] = bbands.bollinger_hband()
        
        return df
    
    def average_true_range(self, window):
        """
        Average true range. Volatility Indicator.
        """
        df = self.technical_indicators
        
        atr = AverageTrueRange(
                high = self.high,
                low = self.low,
                close = self.close,
                window = window,
                fillna = False
                        )
        
        df["Average True Range"] = atr.average_true_range()
        
        return df

# volume indicators
    def on_balance_volume(self):
        """
        On Balance Volume. Volume Indicator.
        """
        df = self.technical_indicators
        
        on_bal_vol = OnBalanceVolumeIndicator(
                                close = self.close,
                                volume = self.volume,
                                fillna = False
                                  )
        
        df["On Balance Volume"] = on_bal_vol.on_balance_volume()
        
        return dataframe
 
    def money_flow_index(self, window):
        """
        Money flow index. Volume Indicator.
        """
        df = self.technical_indicators
        
        mfi = money_flow_index(
                high = self.high,
                low = self.low,
                close = self.close,
                volume = self.volume,
                window = window, 
                fillna = False
                        )
        
        df["Money Flow Index"] = mfi
        
        return dataframe

In [5]:
gspc = dataframe[dataframe["Symbols"] == "^GSPC"]
ixic = dataframe[dataframe["Symbols"] == "^IXIC"]

symbol = TechnicalIndicators(
                        gspc, 
                        gspc["Date"], 
                        gspc["Close"], 
                        gspc["High"], 
                        gspc["Low"], 
                        gspc["Volume"]
                            )

symbol.set_technical_indicators()
symbol.rsi(14)
symbol.bollinger_bands(20)
symbol.rate_change(30).tail(40)
symbol.rate_change(1)
symbol.stochastic_oscillator(14)
symbol.WilliamsR(14)
symbol.on_balance_volume()
symbol.ema(14)
symbol.wma(14)
symbol.money_flow_index(14)
symbol.average_true_range(14)

,Date,Close,High,Low,Volume,RSI,Bollinger Bands Middle,Bollinger Bands Lower,Bollinger Bands Upper,ROC,Stochastic Oscillator,Williams %R Indicator,On Balance Volume,Exponential moving average,Weighted moving average,Money Flow Index,Average True Range
0,2020-09-16,"3,385.49","3,428.92","3,384.45",4710030000,nan,nan,nan,nan,nan,nan,nan,4710030000,nan,nan,nan,0.00
1,2020-09-17,"3,357.01","3,375.17","3,328.82",4371940000,nan,nan,nan,nan,-0.84,nan,nan,338090000,nan,nan,nan,0.00
2,2020-09-18,"3,319.47","3,362.27","3,292.40",7068700000,nan,nan,nan,nan,-1.12,nan,nan,-6730610000,nan,nan,nan,0.00
3,2020-09-21,"3,281.06","3,285.57","3,229.10",4828350000,nan,nan,nan,nan,-1.16,nan,nan,-11558960000,nan,nan,nan,0.00
4,2020-09-22,"3,315.57","3,320.31","3,270.95",3963300000,nan,nan,nan,nan,1.05,nan,nan,-7595660000,nan,nan,nan,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2021-09-09,"4,493.28","4,529.90","4,492.07",3035300000,55.22,"4,485.71","4,407.07","4,564.36",-0.46,62.19,-37.81,90972260000,"4,499.97","4,513.70",56.75,29.14
248,2021-09-10,"4,458.58","4,520.47","4,457.66",2851140000,47.55,"4,485.60","4,406.81","4,564.39",-0.77,8.68,-91.32,88121120000,"4,494.45","4,507.62",49.72,31.55
249,2021-09-13,"4,468.73","4,492.99","4,445.70",3096390000,49.75,"4,485.64","4,406.87","4,564.40",0.23,23.00,-77.00,91217510000,"4,491.02","4,502.74",42.26,32.67
250,2021-09-14,"4,443.05","4,485.68","4,435.46",2568730000,44.65,"4,484.14","4,403.36","4,564.92",-0.57,6.88,-93.12,88648780000,"4,484.62","4,494.53",35.17,33.93


In [9]:
class MovingAverageStrategy:
    """
    Moving Average Crossover Strategy.
    """

    end_date = datetime.today()
    start_date = datetime(end.year-1, end.month, end.day)
    short_window = 20
    long_window = 50
    moving_avg = "SMA" # can only be "SMA" or "EMA" due to if/elif

    def __init__(self, symbol, start_date, end_date, short_window, long_window, moving_avg): 
        """
        Initialize the attributes of a class.
        """
        symbol = self.symbol
        start_date = self.start_date
        end_date = self.end_date
        short_window = self.short_window
        long_window = self.long_window
        moving_avg = self.moving_avg
        
        self.moving_average_strategy = None
        
    def setup(self):
        self.moving_average_strategy = pd.DataFrame()
        self.moving_average_strategy = df

    def crossover_strategy(moving_avg):
        """
        Simple crossover strategy.
        """
        s_col = str(short_window) + " " + moving_avg
        l_col = str(long_window) + " " + moving_avg

        if moving_avg == "SMA":
            # simple moving average column             
            df[s_col] = df["Close"].rolling(window = short_window, min_periods = 1).mean()
            df[l_col] = df["Close"].rolling(window = long_window, min_periods = 1).mean()
        
        elif moving_avg == "EMA":
            # exponential moving average column             
            df[s_col] = df["Close"].ewm(span = short_window, adjust = False).mean()
            df[l_col] = df["Close"].ewm(span = long_window, adjust = False).mean()

        df["Signal"] = 0
        df["Signal"] = np.where(df[s_col] > df[l_col], 1.0, 0.0)
        df["Position"] = df["Signal"].diff()

In [8]:
gspc = dataframe[dataframe["Symbols"] == "^GSPC"]
ixic = dataframe[dataframe["Symbols"] == "^IXIC"]



s = MovingAverageStrategy()

AttributeError: 'MovingAverageStrategy' object has no attribute 'symbol'